<a href="https://colab.research.google.com/github/mdakkon/QuantMethodsFinance/blob/main/NOTEBOOKS/2_Interest_rates_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Methods in Finance

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantMethodsFinance/blob/main/NOTEBOOKS/2_Interest_rates_basics.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.

Clone the repository

In [ ]:
from IPython.display import clear_output

!rm -r {'QuantMethodsFinance'}

!git clone https://github.com/Lolillosky/QuantMethodsFinance

import sys
sys.modules.pop
sys.path.insert(0,'QuantMethodsFinance/CODE')

clear_output()

Import some libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import minimize
import pandas as pd
import re
from scipy.interpolate import CubicSpline


Modify PATH variable depending on how you run the notebook

In [ ]:
# To be used if notebook is run in Google Colab
PATH = '/content/QuantMethodsFinance/'
# To be used if notebook is run in a local clone of the repository
# PATH = '../'


## Interest rate

Given the following:

| Concept | Amount |
|----------|----------|
| Initial investment    | 100     |
| Final value    | 150     |
| T-t (years)    | 5     |

Calculate simple, compounded and continuously compounded interest


In [ ]:
# Your code here

## Continuously compounded interest

Check numerically that:

$$\lim_{n \to \infty}\left(1+r\frac{T-t}{n}\right)^n=\exp\left(r\left(T-t\right)\right)$$

Assume $r=0.03$ $T-t=5$

In [ ]:
# Your code here


## Internal rate of return / yield of a bond

The following code is a simplified version of the pricing of a bond forgeting about dates and day count conventions.

In [ ]:

from enum import Enum

class freq_options(Enum):
    QUARTERLY = 1
    SEMI_ANNUAL = 2
    ANUAL = 3

#TTM : time to maturity
def bond_price(TTM, coupon, yield_to_mat, freq):
  """
  calculetes the bond price given the parameters
  """

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    return np.sum(coupon * delta * (1.0 + yield_to_mat)**(-payment_times)) + (1.0 + yield_to_mat)**(-TTM)


In [ ]:
bond_price(5,0.03,0.05,freq_options.ANUAL)

0.9134104665873835

For a 5 year maturity bond, with annual coupons of $3\%$, plot the dirty price as a function of its yield. If the yield of a bond rises, does the bond holder gain or loose money?

In [ ]:
# Your code here

maturity = 30
coupon = 0.03
ytm = np.linspace(0,0.1,100)

price = [bond_price(maturity, coupon, x, freq_options.ANUAL) for x in ytm]


plt.plot(ytm, price)
plt.plot(0.03, 1.0, 'o', color = 'red')

For yields equal to $1\%,\ 3\%,\ 5\%$, plot the dirty price as time goes by and we approach the maturity of the bond.

In [ ]:
# Your code here

maturity = np.linspace(0,30,1000)
coupon = 0.03
y1 = 0.01
y2 = 0.03
y3 = 0.05


price1 = [bond_price(m, coupon, y1, freq_options.ANUAL) for m in maturity]
price2 = [bond_price(m, coupon, y2, freq_options.ANUAL) for m in maturity]
price3 = [bond_price(m, coupon, y3, freq_options.ANUAL) for m in maturity]


plt.plot(maturity[::-1], price1, label= '0.01')
plt.plot(maturity[::-1], price2, label= '0.03')
plt.plot(maturity[::-1], price3, label= '0.05')

plt.legend()




If the price of the bond is $103\%$, compute its yield. Use fsolve function of Scipy.

In [ ]:
# Your code here

fsolve(lambda y: bond_price(5,0.03,y,freq_options.ANUAL)-1.03, 0.03)

array([0.02356917])

In [ ]:
(lambda x: x+1)(7)



8

## Duration

Code a function to compute the duration of a bond for a given maturity, coupon, yield and payment frequency.

In [ ]:
def bond_duration(TTM, coupon, yield_to_mat, freq, charts = False):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    disc_cf = coupon * delta * (1+yield_to_mat)**(-payment_times)

    disc_cf[-1] += (1+yield_to_mat)**(-payment_times[-1])

    disc_cf_times_t = disc_cf * payment_times

    duration = np.sum(disc_cf_times_t) / np.sum(disc_cf)

    if charts:
      plt.bar(payment_times, disc_cf)
      plt.vlines(duration, 0, 1, color = 'red')
      plt.show()

    return duration



    # your code here

For a 30 year bond with a coupon of $3\%$, plot the bond duration as time evolves for yields of $1\%,\ 3\%,\ 5\%$.

In [ ]:
bond_duration(30,0.03,0.03,freq_options.ANUAL, True)

In [ ]:
maturity = np.linspace(0.0001,30,1000)
coupon = 0.03
y1 = 0.01
y2 = 0.03
y3 = 0.05


duration1 = [bond_duration(m, coupon, y1, freq_options.ANUAL) for m in maturity]
duration2 = [bond_duration(m, coupon, y2, freq_options.ANUAL) for m in maturity]
duration3 = [bond_duration(m, coupon, y3, freq_options.ANUAL) for m in maturity]


plt.plot(maturity[::-1], duration1, label= '0.01')
plt.plot(maturity[::-1], duration2, label= '0.03')
plt.plot(maturity[::-1], duration3, label= '0.05')

plt.legend()



In [ ]:
# Your code here

Same for a zero coupon bond (bond with coupon equal to $0\%$).

In [ ]:
# Your code here

Can you provide a physical interpretation of duration?

## Bootstrapping the yield curve

Let us read the historical data included in the spgb_hist.csv file.

In [ ]:
spgb_hist = pd.read_csv(PATH + 'DATA/spgb_hist.csv', parse_dates=True, index_col=0, dayfirst=True)

spgb_hist.head()

The following code extracts the maturity and coupon of every bond. We will bootstrap the curve on the last date of the dataframe.

In [ ]:

# Calibration date will be last date of the dataframe.
calibration_date = spgb_hist.index[-1]

# Regular expression pattern to extract dates in the format mm/dd/yyyy.
date_pattern =  r"\b\d*\.?\d{0,2}\s(\d{1,2}/\d{1,2}/\d{4})\b"

maturity_dates = [re.search(date_pattern, item).group(1) for item in spgb_hist.columns if re.search(date_pattern, item)]

maturity_dates = pd.to_datetime(maturity_dates, format='%m/%d/%Y');

# Regular expression pattern to extract coupons in the format d.dd.
coupon_pattern = r"\b\d+(\.\d{1,2})?\b"

coupons = np.float64([re.search(coupon_pattern, item).group() for item in spgb_hist.columns if re.search(coupon_pattern, item)])/100

# Year fraction to maturity of each bond.
years_to_maturity = (maturity_dates - calibration_date).days / 365

# Let us print coupons and years to maturity.
print(coupons)
print(years_to_maturity)



The following function calculates the yield of a bond from its coupon, maturity and price.

In [ ]:
def bond_yield(TTM, coupon, price, freq):

    return fsolve(lambda x: bond_price(TTM, coupon, x, freq) - price, coupon,xtol=1e-8)[0]

Let us check that the function works.

In [ ]:
coupon = 0.05
price = 1.00
maturity = 30

bond_yield(maturity, coupon, price, freq_options.ANUAL)

0.05

The following class is the IR curve class. It wrapps a cubic spline interpolator and assumes cubic spline interpolation in zero copupon rates in rates * time (an ussual assumption).

In [ ]:
class IR_Curve:

    def __init__(self, time_pillars, rates):
        # Append zero at the beginning for both time_pillars and rates
        time_pillars = np.concatenate(([0], time_pillars))
        rates = np.concatenate(([0], rates))
        time_times_rate = rates * time_pillars

        # Natural cubic spline interpolation
        self.interpolator = CubicSpline(time_pillars, time_times_rate, bc_type='natural')

    def zero_coupon_rates(self, time_pillars):
        # Check for negative maturities and raise an error if found
        if np.any(time_pillars < 0):
            raise ValueError("Maturity cannot be negative.")

        # Calculate the true limit for zero-coupon rate as t approaches 0
        zero_coupon_rates = np.zeros_like(time_pillars, dtype=float)
        non_zero_mask = time_pillars != 0

        # Calculate rates for non-zero times
        zero_coupon_rates[non_zero_mask] = self.interpolator(time_pillars[non_zero_mask]) / time_pillars[non_zero_mask]

        # Use the derivative at t = 0 as the limit for the zero-coupon rate at t = 0
        zero_coupon_rates[~non_zero_mask] = self.interpolator.derivative()(0)

        return zero_coupon_rates

    def discount_factors(self, time_pillars):
        # Check for negative maturities and raise an error if found
        if np.any(time_pillars < 0):
            raise ValueError("Maturity cannot be negative.")

        # Evaluate the spline and compute discount factors
        return np.exp(-self.interpolator(time_pillars))



In [ ]:
pillars = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30, 40, 50])
rates = np.array([0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.06])
rates = np.ones_like(pillars) * (np.log(1.05))
ir_curve = IR_Curve(pillars, rates)

time_pillars = np.linspace(0.0, 50, 100)


plt.plot(time_pillars, ir_curve.zero_coupon_rates(time_pillars))
plt.plot(pillars, rates, 'o', color='red')


Taking into consideration the curve object, modify the bond price function so that it prices bonds discounting its cashflows with the curve.

In [ ]:
def bond_price_curve(TTM, coupon, curve, freq):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    # your code here
    discount_factors = curve.discount_factors(payment_times)

    bond_price = np.sum(discount_factors * delta * coupon)

    bond_price += discount_factors[-1]

    return bond_price

In [ ]:
bond_price_curve(5, 0.05, ir_curve, freq_options.ANUAL)

Write a function that computes the yield of a bond if the cashflows are discounted with the curve.

In [ ]:
def bond_yield_curve(TTM, coupon, curve, freq):

    # your code
    model_price = bond_price_curve(TTM, coupon, curve, freq)

    return bond_yield(TTM,coupon,model_price,freq)

In [ ]:
bond_yield_curve(5,0.05,ir_curve,freq_options.ANUAL)

0.05

In [ ]:
yields_test = [bond_yield_curve(m,c,ir_curve,freq_options.ANUAL) for m, c in zip(years_to_maturity,coupons) ]

plt.plot(years_to_maturity,yields_test, 'o')

plt.plot(time_pillars, ir_curve.zero_coupon_rates(time_pillars))
plt.plot(pillars, rates, 'o', color='red')


Complete the class below so that it calibrates th zero coupon curve from bond yields

In [ ]:
class yield_curve_bootsrap:

    def __init__(self, bond_maturities, bond_yields, bond_coupons, curve_pillars, init_guess=0.05):

        self.bond_maturities = bond_maturities
        self.bond_yields = bond_yields
        self.curve_pillars = curve_pillars
        self.bond_coupons = np.array(bond_coupons)
        self.init_guess = init_guess*np.ones_like(curve_pillars)

    def compute_model_yields(self, curve_rates):

        # your code here

        curve = IR_Curve(self.curve_pillars, curve_rates)

        return np.array([bond_yield_curve(m,c,curve ,
            freq_options.ANUAL) for m, c in zip(self.bond_maturities,self.bond_coupons) ])

    def compute_yield_mse(self, curve_rates):

        model_yields = self.compute_model_yields(curve_rates)

        return np.mean((model_yields - self.bond_yields)**2)

    def bootstrap(self):

        pillar_rates = minimize(self.compute_yield_mse, self.init_guess)['x']

        return {'pillar rates': pillar_rates, 'curve object': IR_Curve(self.curve_pillars, pillar_rates)}



In [ ]:
boot = yield_curve_bootsrap(years_to_maturity,spgb_hist.iloc[-1]/100, coupons,pillars)



In [ ]:
boot_output = boot.bootstrap()

In [ ]:
plt.plot(time_pillars, boot_output['curve object'].zero_coupon_rates(time_pillars), label = 'zero curve')
plt.plot(boot.bond_maturities, boot.bond_yields, 'o', label = 'market yields')
plt.plot(boot.bond_maturities, boot.compute_model_yields(boot_output['pillar rates']), 'o', label = 'model yields', markersize = 3)
plt.plot(boot.curve_pillars, boot_output['pillar rates'], 'X', label = 'CURVE PILLAR RATES', markersize = 10)


plt.legend()


Plot 1 yr spanning forward rates

In [ ]:
# your code here


Plot the instantaneous forward curve

In [ ]:
# your code here

## Bootstrapping the OIS curve

Read the historical data.

In [ ]:
swaps_hist = pd.read_csv(PATH + 'DATA/Swaps_Hist.csv', parse_dates=True, index_col=0, dayfirst=True, sep = ';')

swaps_hist.head()

In [ ]:
quotes = swaps_hist.loc['11-08-2024']


regex =  r".*?(\d{1,2})Y$"

# Extract ois quotes
ois_quotes = np.array([[int(re.search(regex, s).group(1)), v/100]  for s, v in zip(swaps_hist.columns, quotes.values) if (re.search(regex, s) and 'ESTER' in s)])

# Extract irs quotes
irs_quotes = np.array([[int(re.search(regex, s).group(1)), v/100]  for s, v in zip(swaps_hist.columns, quotes.values) if (re.search(regex, s) and 'EURIBOR' in s)])

plt.plot(ois_quotes[:,0], ois_quotes[:,1], 'o-', label = 'OIS')
plt.plot(irs_quotes[:,0], irs_quotes[:,1], 'o-', label = 'IRS 6M')

plt.legend()


Code a function that returns the par swap rate of an OIS given a curve object.

In [ ]:
def ois_par_rate_curve(TTM, coupon, curve, freq):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    # your code here


Modify the yield curve bootstrapper, so that it calibrates the OIS curve

In [ ]:
# your code here